In [22]:
import pandas as pd

# List of files to process
files = ['/Users/maximbortnik/Desktop/ALL/clustered_deposit_1_month_filtered_data_with_clusters_modified.xlsx',
 '/Users/maximbortnik/Desktop/ALL/clustered_deposit_1_year_filtered_data_with_clusters_modified.xlsx',
 '/Users/maximbortnik/Desktop/ALL/clustered_deposit_3_months_filtered_data_with_clusters_modified.xlsx',
 '/Users/maximbortnik/Desktop/ALL/clustered_deposit_3_years_filtered_data_with_clusters_modified.xlsx',
 '/Users/maximbortnik/Desktop/ALL/clustered_deposit_6_months_filtered_data_with_clusters_modified.xlsx',
 '/Users/maximbortnik/Desktop/ALL/clustered_loan_1_year_filtered_data_with_clusters_modified.xlsx',
 '/Users/maximbortnik/Desktop/ALL/clustered_loan_3_years_filtered_data_with_clusters_modified.xlsx',
 '/Users/maximbortnik/Desktop/ALL/clustered_loan_5_years_filtered_data_with_clusters_modified.xlsx',
 '/Users/maximbortnik/Desktop/ALL/clustered_loan_10_years_filtered_data_with_clusters_modified.xlsx']

# Process each file
for file in files:
    # Read the Excel file
    df = pd.read_excel(file)
    
    # Drop the last two columns
    df = df.iloc[:, :-2]
    
    # Save the modified dataframe to a new Excel file
    output_file = file.replace(".xlsx", "_modified.xlsx")
    df.to_excel(output_file, index=False)

output_files = [file.replace(".xlsx", "_modified.xlsx") for file in files]
output_files


['/Users/maximbortnik/Desktop/ALL/clustered_deposit_1_month_filtered_data_with_clusters_modified_modified.xlsx',
 '/Users/maximbortnik/Desktop/ALL/clustered_deposit_1_year_filtered_data_with_clusters_modified_modified.xlsx',
 '/Users/maximbortnik/Desktop/ALL/clustered_deposit_3_months_filtered_data_with_clusters_modified_modified.xlsx',
 '/Users/maximbortnik/Desktop/ALL/clustered_deposit_3_years_filtered_data_with_clusters_modified_modified.xlsx',
 '/Users/maximbortnik/Desktop/ALL/clustered_deposit_6_months_filtered_data_with_clusters_modified_modified.xlsx',
 '/Users/maximbortnik/Desktop/ALL/clustered_loan_1_year_filtered_data_with_clusters_modified_modified.xlsx',
 '/Users/maximbortnik/Desktop/ALL/clustered_loan_3_years_filtered_data_with_clusters_modified_modified.xlsx',
 '/Users/maximbortnik/Desktop/ALL/clustered_loan_5_years_filtered_data_with_clusters_modified_modified.xlsx',
 '/Users/maximbortnik/Desktop/ALL/clustered_loan_10_years_filtered_data_with_clusters_modified_modified.x

In [41]:
import pandas as pd
import os

# Define a function to process each file
def process_file(file_path, group_by_fields, min_rate_col, max_rate_col):
    df = pd.read_excel(file_path)
    
    # Calculate mean rates
    mean_rates = df.groupby(group_by_fields).agg(
        {min_rate_col: 'mean', max_rate_col: 'mean'}
    ).reset_index()
    
    # Calculate first rates
    first_rates = df.groupby(group_by_fields).agg(
        {min_rate_col: 'first', max_rate_col: 'first'}
    ).reset_index()
    
    # Calculate last rates
    last_rates = df.groupby(group_by_fields).agg(
        {min_rate_col: 'last', max_rate_col: 'last'}
    ).reset_index()
    
    # Merge mean, first, and last rates
    merged_df = pd.merge(mean_rates, first_rates, on=group_by_fields, suffixes=('_mean', '_first'))
    merged_df = pd.merge(merged_df, last_rates, on=group_by_fields)
    
    # Calculate rate changes
    merged_df['Min rate change'] = merged_df[f'{min_rate_col}_first'] - merged_df[f'{min_rate_col}_mean']
    merged_df['Max rate change'] = merged_df[f'{max_rate_col}_first'] - merged_df[f'{max_rate_col}_mean']
    
    # Rename columns for last rates
    merged_df.rename(columns={min_rate_col: f'{min_rate_col}_last', max_rate_col: f'{max_rate_col}_last'}, inplace=True)
    
    # Select relevant columns to save
    result = merged_df[group_by_fields + ['Min rate change', 'Max rate change', f'{min_rate_col}_first', f'{max_rate_col}_first', f'{min_rate_col}_last', f'{max_rate_col}_last']]
    
    return result


# List of file paths
file_paths = [
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/deposit_1_month_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/deposit_1_year_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/deposit_3_months_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/deposit_3_years_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/deposit_6_months_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/loan_1_year_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/loan_3_years_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/loan_5_years_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/loan_10_years_filtered_data_withOUT_clusters.xlsx'
]

# Fields to group by and rate columns for deposits and loans
deposit_fields = ['Название банка', 'Капитализация', 'Пополнение', 'Минимальный депозит']
loan_fields = ['Название банка', 'Минимальный возраст', 'Минимальная сумма', 'Максимальная сумма']
min_rate_col = 'Минимальная ставка'
max_rate_col = 'Максимальная ставка'

# Process each file and save the results
for file_path in file_paths:
    if 'deposit' in file_path:
        group_by_fields = deposit_fields
    elif 'loan' in file_path:
        group_by_fields = loan_fields
    
    result = process_file(file_path, group_by_fields, min_rate_col, max_rate_col)
    
    # Save the result to a new file
    base_name = os.path.basename(file_path)
    new_file_path = f'/Users/maximbortnik/Desktop/ALL/FINAL DATA/1average_rates_{base_name}'
    result.to_excel(new_file_path, index=False)
    print(f'Saved processed data to {new_file_path}')


Saved processed data to /Users/maximbortnik/Desktop/ALL/FINAL DATA/1average_rates_deposit_1_month_filtered_data_withOUT_clusters.xlsx
Saved processed data to /Users/maximbortnik/Desktop/ALL/FINAL DATA/1average_rates_deposit_1_year_filtered_data_withOUT_clusters.xlsx
Saved processed data to /Users/maximbortnik/Desktop/ALL/FINAL DATA/1average_rates_deposit_3_months_filtered_data_withOUT_clusters.xlsx
Saved processed data to /Users/maximbortnik/Desktop/ALL/FINAL DATA/1average_rates_deposit_3_years_filtered_data_withOUT_clusters.xlsx
Saved processed data to /Users/maximbortnik/Desktop/ALL/FINAL DATA/1average_rates_deposit_6_months_filtered_data_withOUT_clusters.xlsx
Saved processed data to /Users/maximbortnik/Desktop/ALL/FINAL DATA/1average_rates_loan_1_year_filtered_data_withOUT_clusters.xlsx
Saved processed data to /Users/maximbortnik/Desktop/ALL/FINAL DATA/1average_rates_loan_3_years_filtered_data_withOUT_clusters.xlsx
Saved processed data to /Users/maximbortnik/Desktop/ALL/FINAL DATA/1

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import os

# Function to determine the optimal number of clusters using Elbow Method and Silhouette Score
def find_optimal_clusters(data, max_k):
    iters = range(2, max_k+1)
    
    sse = []
    silhouette_scores = []
    
    for k in iters:
        kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
        kmeans.fit(data)
        sse.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(data, kmeans.labels_))
    
    # Plot SSE for Elbow Method
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(iters, sse, '-o')
    plt.xlabel('Number of clusters')
    plt.ylabel('SSE')
    plt.title('Elbow Method')
    
    # Plot Silhouette Scores
    plt.subplot(1, 2, 2)
    plt.plot(iters, silhouette_scores, '-o')
    plt.xlabel('Number of clusters')
    plt.ylabel('Silhouette Score')
    plt.title('Silhouette Scores')
    
    plt.show()

# Function to perform K-means clustering and plot the results
def perform_kmeans_clustering(df, n_clusters):
    clustering_data = df[['Min rate change', 'Max rate change']]
    kmeans = KMeans(n_clusters=n_clusters, init='k-means++', random_state=42)
    df['Cluster'] = kmeans.fit_predict(clustering_data)
    
    # Plotting the clusters
    plt.figure(figsize=(8, 6))
    plt.scatter(df['Min rate change'], df['Max rate change'], c=df['Cluster'], cmap='viridis')
    plt.xlabel('Min rate change')
    plt.ylabel('Max rate change')
    plt.title(f'K-means Clustering with {n_clusters} clusters')
    plt.show()
    
    return df

# List of file paths
file_paths = [
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/average_rates_deposit_1_month_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/average_rates_deposit_1_year_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/average_rates_deposit_3_months_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/average_rates_deposit_3_years_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/average_rates_deposit_6_months_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/average_rates_loan_1_year_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/average_rates_loan_3_years_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/average_rates_loan_5_years_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/average_rates_loan_10_years_filtered_data_withOUT_clusters.xlsx'
]

# Process each file to determine the optimal number of clusters
for file_path in file_paths:
    df = pd.read_excel(file_path)
    clustering_data = df[['Min rate change', 'Max rate change']]
    print(f"Finding optimal clusters for {file_path}")
    find_optimal_clusters(clustering_data, 10)

# Optimal number of clusters determined from the above step
optimal_clusters = {
    'average_rates_deposit_1_month_filtered_data_withOUT_clusters.xlsx': 2,
    'average_rates_deposit_1_year_filtered_data_withOUT_clusters.xlsx': 2,
    'average_rates_deposit_3_months_filtered_data_withOUT_clusters.xlsx': 3,
    'average_rates_deposit_3_years_filtered_data_withOUT_clusters.xlsx': 2,
    'average_rates_deposit_6_months_filtered_data_withOUT_clusters.xlsx': 2,
    'average_rates_loan_1_year_filtered_data_withOUT_clusters.xlsx': 3,
    'average_rates_loan_3_years_filtered_data_withOUT_clusters.xlsx': 2,
    'average_rates_loan_5_years_filtered_data_withOUT_clusters.xlsx': 2,
    'average_rates_loan_10_years_filtered_data_withOUT_clusters.xlsx': 2,
}

# Apply K-means clustering and save the results
for file_path in file_paths:
    df = pd.read_excel(file_path)
    base_name = os.path.basename(file_path)
    n_clusters = optimal_clusters[base_name]
    clustered_df = perform_kmeans_clustering(df, n_clusters)
    new_file_path = f'/Users/maximbortnik/Desktop/ALL/FINAL DATA/final_data_{base_name}'
    clustered_df.to_excel(new_file_path, index=False)
    print(f'Saved clustered data to {new_file_path}')


In [42]:
import pandas as pd
import os

# List of file paths
file_paths = [
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/final_data_average_rates_deposit_1_month_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/final_data_average_rates_deposit_1_year_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/final_data_average_rates_deposit_3_months_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/final_data_average_rates_deposit_3_years_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/final_data_average_rates_deposit_6_months_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/final_data_average_rates_loan_1_year_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/final_data_average_rates_loan_3_years_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/final_data_average_rates_loan_5_years_filtered_data_withOUT_clusters.xlsx',
    '/Users/maximbortnik/Desktop/ALL/FINAL DATA/final_data_average_rates_loan_10_years_filtered_data_withOUT_clusters.xlsx'
]

# Process each file and save clusters separately
for file_path in file_paths:
    df = pd.read_excel(file_path)
    base_name = os.path.basename(file_path)
    
    # Ensure 'Cluster' column exists in the dataframe
    if 'Cluster' in df.columns:
        # Get unique cluster labels
        clusters = df['Cluster'].unique()
        
        # Save each cluster to a separate file
        for cluster in clusters:
            cluster_df = df[df['Cluster'] == cluster]
            cluster_file_path = f'/Users/maximbortnik/Desktop/ALL/FINAL DATA/cluster_{int(cluster)}_{base_name}'
            cluster_df.to_excel(cluster_file_path, index=False)
            print(f'Saved clustered data to {cluster_file_path}')
    else:
        print(f"No 'Cluster' column found in {file_path}")


Saved clustered data to /Users/maximbortnik/Desktop/ALL/FINAL DATA/cluster_0_final_data_average_rates_deposit_1_month_filtered_data_withOUT_clusters.xlsx
Saved clustered data to /Users/maximbortnik/Desktop/ALL/FINAL DATA/cluster_1_final_data_average_rates_deposit_1_month_filtered_data_withOUT_clusters.xlsx
Saved clustered data to /Users/maximbortnik/Desktop/ALL/FINAL DATA/cluster_1_final_data_average_rates_deposit_1_year_filtered_data_withOUT_clusters.xlsx
Saved clustered data to /Users/maximbortnik/Desktop/ALL/FINAL DATA/cluster_0_final_data_average_rates_deposit_1_year_filtered_data_withOUT_clusters.xlsx
Saved clustered data to /Users/maximbortnik/Desktop/ALL/FINAL DATA/cluster_0_final_data_average_rates_deposit_3_months_filtered_data_withOUT_clusters.xlsx
Saved clustered data to /Users/maximbortnik/Desktop/ALL/FINAL DATA/cluster_1_final_data_average_rates_deposit_3_months_filtered_data_withOUT_clusters.xlsx
Saved clustered data to /Users/maximbortnik/Desktop/ALL/FINAL DATA/cluster_2